In [ ]:
import pandas as pd
import joblib
import os
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.resnet50 import preprocess_input
from tkinter import Tk



In [26]:

# Definir la función de preprocesamiento de la imagen
def preprocess_image(image_path, size=(256, 256)):
    with Image.open(image_path) as img:
        img = img.resize(size)
        img_array = np.array(img)
        img_array = preprocess_input(img_array)  # Normalización específica de ResNet50
        img_array = np.expand_dims(img_array, axis=0)  # Añadir dimensión para el batch
        return img_array

# Función para predecir con el modelo de imagen
def predict_image(image_path, image_model):
    img = preprocess_image(image_path)
    img_pred = image_model.predict(img)
    return img_pred.flatten()  # Aplanar para que tenga la forma correcta

# Cargar el modelo de imágenes
image_model_path = '/Users/luiseduardogarciablanco/Desktop/bootcamp/Flask_proyecto_cancer/static/model/resnet50_fine_tuned_model_20.h5'
image_model = load_model(image_model_path)

# Ruta a la imagen para la prueba
# Crear una ventana para seleccionar la imagen
Tk().withdraw()  # Ocultar la ventana principal
example_image_path = askopenfilename(title="Selecciona una imagen", filetypes=[("Archivos de imagen", "*.jpg")])
#example_image_path = '/Users/luiseduardogarciablanco/Desktop/nueva data cancer/test web/jpg_ensambled_model/ISIC_5046058.jpg'

# Probar la función de preprocesamiento y predicción de la imagen
preprocessed_image = preprocess_image(example_image_path)
print("Forma de la imagen preprocesada:", preprocessed_image.shape)

image_prediction = predict_image(example_image_path, image_model)
print("Predicción de la imagen:", image_prediction)

Forma de la imagen preprocesada: (1, 256, 256, 3)


2024-09-04 21:21:42.127789: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 1s/step
Predicción de la imagen: [0.9997173]


In [27]:


# Cargar el dataset
df = pd.read_csv("/Users/luiseduardogarciablanco/Desktop/bootcamp/Flask_proyecto_cancer/static/data/dataset_flask.csv")

# Función para extraer el image_id del path
def extract_image_id_from_path(image_path):
    # Extraer el nombre del archivo con extensión
    base_name = os.path.basename(image_path)
    # Obtener el nombre del archivo sin extensión
    image_id = os.path.splitext(base_name)[0]
    return image_id

# Función para predecir con el modelo de metadatos
def predict_metadata(df, metadata_model, image_id):
    # Obtener los metadatos correspondientes al image_id
    metadata = df[df['isic_id'] == image_id].drop(columns=['isic_id', 'target'])
    
    # Predicción del modelo de metadatos
    pred = metadata_model.predict_proba(metadata)[:, 1]  # Probabilidad de la clase positiva
    
    return pred

# Cargar el modelo de metadatos
metadata_model_path = '/Users/luiseduardogarciablanco/Desktop/bootcamp/Flask_proyecto_cancer/static/model/metadata_model.pkl'
metadata_model = joblib.load(metadata_model_path)

# ID de ejemplo para la prueba
example_image_id = extract_image_id_from_path(example_image_path)

# Probar la función de predicción de metadatos
metadata_prediction = predict_metadata(df, metadata_model, example_image_id)
print("Predicción de metadatos:", metadata_prediction)

Predicción de metadatos: [0.75518262]


In [28]:

# Funciones previamente definidas (preprocess_image, predict_image, extract_image_id_from_path, predict_metadata)

def predict_ensemble(image_path, df, image_model, metadata_model, ensemble_model):
    # Obtener el image_id (sin extensión .jpg)
    image_id = extract_image_id_from_path(example_image_path)
    
    # Obtener la predicción de la imagen
    image_pred = predict_image(example_image_path, image_model)
    
    # Obtener la predicción de los metadatos
    metadata_pred = predict_metadata(df, metadata_model, image_id)
    
    # Combinar las predicciones
    features = np.concatenate([image_pred, metadata_pred])
    
    # Realizar la predicción con el modelo ensamblador
    ensemble_pred = ensemble_model.predict([features])
    return ensemble_pred[0]

# Cargar los modelos
image_model_path = '/Users/luiseduardogarciablanco/Desktop/bootcamp/Flask_proyecto_cancer/static/model/resnet50_fine_tuned_model_20.h5'
image_model = load_model(image_model_path)

metadata_model_path = '/Users/luiseduardogarciablanco/Desktop/bootcamp/Flask_proyecto_cancer/static/model/metadata_model.pkl'
metadata_model = joblib.load(metadata_model_path)

ensemble_model_path = '/Users/luiseduardogarciablanco/Desktop/bootcamp/Flask_proyecto_cancer/static/model/ensemble_model.pkl'
ensemble_model = joblib.load(ensemble_model_path)


# Probar la función de predicción del modelo ensamblador
ensemble_prediction = predict_ensemble(example_image_path, df, image_model, metadata_model, ensemble_model)
print("Predicción del modelo ensamblador:", ensemble_prediction)

2024-09-04 21:21:57.585501: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 905ms/step
Predicción del modelo ensamblador: 1
